In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install flair==0.10
!pip install seqeval
!pip install sacremoses

     |████████████████████████████████| 322 kB 14.0 MB/s 
     |████████████████████████████████| 46 kB 4.7 MB/s 
     |████████████████████████████████| 48 kB 6.3 MB/s 
     |████████████████████████████████| 4.2 MB 94.2 MB/s 
     |████████████████████████████████| 981 kB 63.2 MB/s 
     |████████████████████████████████| 84 kB 4.0 MB/s 
     |████████████████████████████████| 1.2 MB 57.4 MB/s 
     |████████████████████████████████| 19.7 MB 58.1 MB/s 
     |████████████████████████████████| 788 kB 81.4 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 6.6 MB 71.3 MB/s 
     |████████████████████████████████| 596 kB 86.4 MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9692 sha256=a021b38697d0fd590b7bcebb3207364b4600216293f336

In [3]:
!pip install fasttext

     |████████████████████████████████| 68 kB 6.3 MB/s 
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3147847 sha256=88837b115a40f5fd6df7cec82fc768ca40ff42641520f4bf7bc479de5e3a32d8
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [4]:
# !unrar x -Y "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/wiki.id.rar" "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/"

In [5]:
import pandas as pd
import re
import numpy as np

In [6]:
ROOT = "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen"
MODEL = "BiLSTM-CRF-fasttext-cbow"
FINAL_MODEL_NAME = "fasttext-cbow"

In [7]:
# !unrar x "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/cc.id.300.bin" "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/"

In [8]:
# !gunzip "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/cc.id.300.bin.gz" "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/"

In [9]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# define columns
columns = {0: 'kata', 1: 'label'}

# this is the folder in which train, test and dev files reside

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(f"{ROOT}/nested", columns,
                              train_file='flair-train.txt',
                              test_file='flair-test.txt',
                              dev_file='flair-test.txt')

2022-05-19 08:46:08,095 Reading data from /gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested
2022-05-19 08:46:08,096 Train: /gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/flair-train.txt
2022-05-19 08:46:08,096 Dev: /gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/flair-test.txt
2022-05-19 08:46:08,098 Test: /gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/flair-test.txt


In [10]:
print(corpus.train[0].to_tagged_string('label'))

Dan <O+O> jika <O+O> orang <O+O> yang <O+O> berhutang <O+O> itu <O+O> dalam <O+O> kesukaran <O+O> maka <O+O> berilah <O+O> tangguh <O+O> sampai <O+O> dia <O+O> berkelapangan <O+O> Dan <O+O> menyedekahkan <O+O> sebagian <O+O> atau <O+O> semua <O+O> utang <O+O> itu <O+O> lebih <O+O> baik <O+O> bagimu <O+O> jika <O+O> kamu <O+O> mengetahui <O+O>


In [11]:
tag_type = 'label'

tag_dictionary = corpus.make_label_dictionary(tag_type)

2022-05-19 08:46:11,917 Computing label dictionary. Progress:


100%|██████████| 763/763 [00:00<00:00, 9253.22it/s]

2022-05-19 08:46:12,005 Corpus contains the labels: label (#29729)
2022-05-19 08:46:12,006 Created (for label 'label') Dictionary with 5 tags: <unk>, O+O, B-PER+B-PER, I-PER+I-PER, O+I-PER


In [12]:
print(tag_dictionary)

Dictionary with 5 tags: <unk>, O+O, B-PER+B-PER, I-PER+I-PER, O+I-PER


In [13]:
import gensim
import fasttext
import fasttext.util

In [14]:
from flair.embeddings import FastTextEmbeddings, WordEmbeddings

import requests, os
session = requests.Session()
session.verify = False
session.trust_env = False
os.environ['CURL_CA_BUNDLE']="" # or whaever other is interfering with 

embedding = WordEmbeddings('id')

2022-05-19 08:46:12,178 https://flair.informatik.hu-berlin.de/resources/embeddings/token/id-wiki-fasttext-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmpbkdn8gqq


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
100%|██████████| 360822128/360822128 [00:11<00:00, 31501483.96B/s]

2022-05-19 08:46:23,737 copying /tmp/tmpbkdn8gqq to cache at /root/.flair/embeddings/id-wiki-fasttext-300d-1M.vectors.npy


2022-05-19 08:46:24,127 removing temp file /tmp/tmpbkdn8gqq
2022-05-19 08:46:24,228 https://flair.informatik.hu-berlin.de/resources/embeddings/token/id-wiki-fasttext-300d-1M not found in cache, downloading to /tmp/tmp1cqbbvtv


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
100%|██████████| 11638719/11638719 [00:00<00:00, 21973894.77B/s]

2022-05-19 08:46:24,860 copying /tmp/tmp1cqbbvtv to cache at /root/.flair/embeddings/id-wiki-fasttext-300d-1M
2022-05-19 08:46:24,876 removing temp file /tmp/tmp1cqbbvtv


In [15]:
# import fasttext.util
# ft = fasttext.load_model('/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/wiki.id.bin')
# ft.get_word_vector('12')

In [16]:
# import requests, os
# session = requests.Session()
# session.verify = False
# session.trust_env = False
# os.environ['CURL_CA_BUNDLE']="" # or whaever other is interfering with 

from flair.models import SequenceTagger
from flair.embeddings import WordEmbeddings, FastTextEmbeddings
tagger : SequenceTagger = SequenceTagger(hidden_size=256,
                                       embeddings=embedding,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True)
print(tagger)

SequenceTagger(
  (embeddings): WordEmbeddings(
    'id'
    (embedding): Embedding(300686, 300)
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=300, out_features=300, bias=True)
  (rnn): LSTM(300, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=7, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)


In [17]:
from flair.trainers import ModelTrainer
trainer : ModelTrainer = ModelTrainer(tagger, corpus)
trainer.train(f'/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/results-model/final-model-{FINAL_MODEL_NAME}',
              learning_rate=0.1,
              max_epochs=200,
              )

2022-05-19 08:46:42,378 ----------------------------------------------------------------------------------------------------
2022-05-19 08:46:42,382 Model: "SequenceTagger(
  (embeddings): WordEmbeddings(
    'id'
    (embedding): Embedding(300686, 300)
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=300, out_features=300, bias=True)
  (rnn): LSTM(300, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=7, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2022-05-19 08:46:42,383 ----------------------------------------------------------------------------------------------------
2022-05-19 08:46:42,385 Corpus: "Corpus: 763 train + 763 dev + 763 test sentences"
2022-05-19 08:46:42,386 ----------------------------------------------------------------------------------------------------
2022-05-19 08:46:42,388 Parameters:
2022-05-19 08:46:42,390  - learning_rate

/usr/local/lib/python3.7/dist-packages/flair/trainers/trainer.py:65: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  "There should be no best model saved at epoch 1 except there is a model from previous trainings"


2022-05-19 08:46:42,877 ----------------------------------------------------------------------------------------------------
2022-05-19 08:46:43,367 epoch 1 - iter 2/24 - loss 0.67150830 - samples/sec: 131.20 - lr: 0.100000
2022-05-19 08:46:43,641 epoch 1 - iter 4/24 - loss 0.50395817 - samples/sec: 234.87 - lr: 0.100000
2022-05-19 08:46:44,015 epoch 1 - iter 6/24 - loss 0.41446495 - samples/sec: 172.08 - lr: 0.100000
2022-05-19 08:46:44,280 epoch 1 - iter 8/24 - loss 0.38474888 - samples/sec: 244.00 - lr: 0.100000
2022-05-19 08:46:44,709 epoch 1 - iter 10/24 - loss 0.35538661 - samples/sec: 149.90 - lr: 0.100000
2022-05-19 08:46:45,028 epoch 1 - iter 12/24 - loss 0.34307792 - samples/sec: 202.19 - lr: 0.100000
2022-05-19 08:46:45,289 epoch 1 - iter 14/24 - loss 0.33783763 - samples/sec: 247.19 - lr: 0.100000
2022-05-19 08:46:45,623 epoch 1 - iter 16/24 - loss 0.32962710 - samples/sec: 193.18 - lr: 0.100000
2022-05-19 08:46:45,923 epoch 1 - iter 18/24 - loss 0.32187467 - samples/sec: 2

{'dev_loss_history': [tensor(0.2614, device='cuda:0'),
  tensor(0.2201, device='cuda:0'),
  tensor(0.1886, device='cuda:0'),
  tensor(0.1634, device='cuda:0'),
  tensor(0.1273, device='cuda:0'),
  tensor(0.1155, device='cuda:0'),
  tensor(0.1006, device='cuda:0'),
  tensor(0.1067, device='cuda:0'),
  tensor(0.0950, device='cuda:0'),
  tensor(0.0874, device='cuda:0'),
  tensor(0.0876, device='cuda:0'),
  tensor(0.0872, device='cuda:0'),
  tensor(0.0655, device='cuda:0'),
  tensor(0.0699, device='cuda:0'),
  tensor(0.0733, device='cuda:0'),
  tensor(0.0775, device='cuda:0'),
  tensor(0.0595, device='cuda:0'),
  tensor(0.0574, device='cuda:0'),
  tensor(0.0560, device='cuda:0'),
  tensor(0.0733, device='cuda:0'),
  tensor(0.0582, device='cuda:0'),
  tensor(0.0533, device='cuda:0'),
  tensor(0.0526, device='cuda:0'),
  tensor(0.0591, device='cuda:0'),
  tensor(0.0542, device='cuda:0'),
  tensor(0.0565, device='cuda:0'),
  tensor(0.0625, device='cuda:0'),
  tensor(0.0500, device='cuda:0'),


In [18]:
from flair.data import Sentence
from flair.models import SequenceTagger
# load the trained model
model = SequenceTagger.load(f'/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/results-model/final-model-{FINAL_MODEL_NAME}/best-model.pt')

2022-05-19 09:31:20,773 loading file /gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/results-model/final-model-fasttext-cbow/best-model.pt


In [19]:
# create example sentence
sentence = Sentence('yaitu mereka orang-orang yang dimurkai allah')
# predict the tags
model.predict(sentence)

print(sentence.to_tagged_string())

yaitu <O+O> mereka <O+O> orang-orang <B-PER+B-PER> yang <I-PER+I-PER> dimurkai <I-PER+I-PER> allah <I-PER+I-PER>


In [20]:
for entity in sentence.get_spans('label'):
    print(entity)

Span [1]: "yaitu"   [− Labels: O+O (0.8865)]
Span [2]: "mereka"   [− Labels: O+O (0.8905)]
Span [3,4,5,6]: "orang-orang yang dimurkai allah"   [− Labels: PER+I-PER (0.8444)]


In [21]:
sentence.to_dict('label')

{'entities': [{'end_pos': 5,
   'labels': [O+O (0.8865)],
   'start_pos': 0,
   'text': 'yaitu'},
  {'end_pos': 12, 'labels': [O+O (0.8905)], 'start_pos': 6, 'text': 'mereka'},
  {'end_pos': 44,
   'labels': [PER+I-PER (0.8444)],
   'start_pos': 13,
   'text': 'orang-orang yang dimurkai allah'}],
 'labels': [],
 'text': 'yaitu mereka orang-orang yang dimurkai allah'}

In [22]:
from ast import literal_eval

test_data = pd.read_csv(f"{ROOT}/nested/test_data.csv")

test_data['sent'] = test_data['sent'].apply(literal_eval)
test_data['tag'] = test_data['tag'].apply(literal_eval)
test_data['pos'] = test_data['pos'].apply(literal_eval)

ii_lab = []
for i, d in test_data.iterrows() :
  sent = ' '.join(d.sent)

  sentence = Sentence(sent)

  model.predict(sentence)

  t = sentence.to_tagged_string()

  word = [i.strip() for i in re.split('\<(.*?)\>', t)][:-1][::2] 
  tag = [i.strip() for i in re.split('\<(.*?)\>', t)][:-1][1::2] 
  
  ii_lab.append(tag)


In [23]:
test_data['pred'] = ii_lab
y_pred = ii_lab
y_true = test_data['tag'].to_list()

In [24]:
def get_level_token(seqs, max_lev=2, level=0) : 
  new_all_level = []
  for seq in seqs : 
    new_all_level_i = []
    for tag in seq : 
      new_tag = []
      tag_per_level = tag.split('+')
      for lev in range(max_lev) :
        if lev + 1 > len(tag_per_level) : 
          new_tag.append(tag_per_level[-1])
        else : 
          new_tag.append(tag_per_level[lev])
      new_all_level_i.append(new_tag)
    new_all_level.append(new_all_level_i)
  
  new_seq = []
  for seq in new_all_level :
    seq_i = [] 
    for tag in seq : 
      seq_i.append(tag[level])
    new_seq.append(seq_i)
  
  return new_seq

y_true_0 = get_level_token(y_true, max_lev=2, level=0)
y_pred_0 = get_level_token(y_pred, max_lev=2, level=0)

In [25]:
from seqeval.metrics import classification_report
print(classification_report(y_true_0, y_pred_0))

              precision    recall  f1-score   support

         PER       0.79      0.74      0.76       222

   micro avg       0.79      0.74      0.76       222
   macro avg       0.79      0.74      0.76       222
weighted avg       0.79      0.74      0.76       222



In [26]:
y_true_1 = get_level_token(y_true, max_lev=2, level=1)
y_pred_1 = get_level_token(y_pred, max_lev=2, level=1)

from seqeval.metrics import classification_report
print(classification_report(y_true_1, y_pred_1))

              precision    recall  f1-score   support

         PER       0.76      0.71      0.74       222

   micro avg       0.76      0.71      0.74       222
   macro avg       0.76      0.71      0.74       222
weighted avg       0.76      0.71      0.74       222



In [27]:
y_test_token = []
for i in y_true : 
  y_test_token = y_test_token + i

y_pred_token = []
for i in y_pred : 
  y_pred_token = y_pred_token + i

from sklearn.metrics import classification_report
print(classification_report(y_test_token, y_pred_token))


              precision    recall  f1-score   support

 B-PER+B-PER       0.88      0.82      0.85       222
 I-PER+I-PER       0.82      0.73      0.77       444
     O+I-PER       0.61      0.10      0.17       114
         O+O       0.96      0.99      0.98      6243

    accuracy                           0.95      7023
   macro avg       0.82      0.66      0.69      7023
weighted avg       0.95      0.95      0.95      7023

